# Is the rise of house prices in the UK due to a shortage of new house builds?

### parameters
- *study period*: 2010–2020
- *control period*: –2010

### assumptions
__potential variables__:
- *demand*: population growth
- *ability to buy*: economic growth, umemployment rate, average wage, interest rate, etc.
- *incentive to build*: house prices, ?
- *market composition*: public vs. private builders

### hypothesis
`house prices rose in the UK in 2010–2020 above “normal” levels due to fewer houses built, which in turn was due to the elimination of public builders`

### required data
all broken by geographic region
- population growth
- economic growth
- unemployment
- interest rate
- house prices
- house builds (by builder)
- house sales

In [21]:
import pandas as pd

In [22]:
def remove_newlines(df):
    df.columns = df.columns.str.replace('\n', ' ')
    return df

def clean_builds(file):
    df = pd.read_csv(file, index_col=0, header=5)
    df = df.drop(columns=df.columns[:1])
    df = remove_newlines(df)
    df['Year'] = df.index.str.replace(r'.* ', '')
    df = df.groupby("Year").sum()
    return df

def clean_sales(file):
    df = pd.read_csv(file, index_col=0, header=6)
    df = remove_newlines(df)
    df = df.dropna(axis=1)
    df = df.groupby('Region/Country name').sum()
    df.columns = df.columns.str.replace('Year ending ', '')
    df = df.transpose()
    df = df.drop(columns='Wales')
    df = df[df.index.str.match(r'Dec \d{4}')]
    df['Year'] = df.index.str.replace(r'[A-Z][a-z]{2} ', '')
    df = df.groupby('Year').sum()
    return df

sales_existing = clean_sales('data/hpssa_23_10.csv')
sales_new = clean_sales('data/hpssa_22_10.csv')
builds = clean_builds('data/housebuilding_4.csv')

population = pd.read_csv('data/series-140420.csv', index_col=0, header=7).iloc[:, 0]
population.index = population.index.astype('str')
pop_growth = pd.Series(population.values[1:]-population.values[:-1], index=population.index[1:])
pop_growth.name = "Pop_Growth"


In [23]:
rs = [str(i) for i in range(1995, 2018)]
rs2 = [str(i) for i in range(1978, 2018)]
new = sales_new.sum(axis=1)
build = builds['All Dwellings.1']
exist = sales_existing.sum(axis=1)
new.name = "New"
exist.name = "Existing"
build.name = "Built"
# print(((pd.DataFrame([build[rs], pop_growth[rs], exist[rs], new[rs]])/1000).astype('int32').transpose()).plot())
# print(((pd.DataFrame([build[rs2], pop_growth[rs2]])/1000).astype('int32').transpose()).plot())
# print(((pd.DataFrame([pop_growth[rs2]/build[rs2]])).transpose()).plot())
pop_growth/build

1972         NaN
1973         NaN
1974         NaN
1975         NaN
1976         NaN
1977         NaN
1978   -0.049314
1979    0.296477
1980    0.442229
1981    0.165064
1982   -0.455444
1983    0.148978
1984    0.533668
1985    0.881779
1986    0.761916
1987    0.674183
1988    0.579949
1989    0.915119
1990    0.982246
1991    1.301423
1992    1.013697
1993    0.875271
1994    0.958355
1995    1.035382
1996    0.936347
1997    1.002007
1998    1.126533
1999    1.485710
2000    1.493079
2001    1.752124
2002    1.847222
2003    1.881161
2004    2.036087
2005    2.903104
2006    2.572423
2007    2.785326
2008    3.409905
2009    3.494159
2010    4.675349
2011    4.609313
2012    3.632667
2013    3.661032
2014    4.168223
2015    3.601909
2016    3.792642
2017    2.412626
2018    2.405256
2019         NaN
dtype: float64

### Conclusion (preliminary):
`while house building did decline, new demand far outstripped demand (population growth rate was 2-4 times that in the period before 1998)`

*the hypothesis is presumed to be __false__*

---

### Further investigation:

Confirm conclusion for submarkets, regions

In [24]:
def clean_pop(file):
    df = pd.read_csv(file, header=4)
    if str(df['Year/Code'].dtype ) == 'object':
        df.loc[df.index[~df['Year/Code'].str.match('\d{4}').fillna(False)], 'Year/Code'] = 0
    else:
        df['Year/Code'] = df['Year/Code'].fillna(0)
    df['Year/Code'] = df['Year/Code'].astype('int32')
    def extend_year(r):
        r['Year/Code'] = df['Year/Code'][:r.name].max()
        return r
    df = df.apply(extend_year, axis=1)
    df = df[df['Standard Statistical Regions'].notna()]
    df = df.set_index('Year/Code')
    df = df[['Standard Statistical Regions', 'All Persons']]
    df.index.name = 'Year'
    df.columns = ['Region', 'Pop']
    df = df.pivot(columns='Region', values='Pop').astype('int32')
    df = df.drop(columns='Wales')
    return df

df1 = clean_pop('data/pop_3.csv')
pop_78 = df1
df1 = df1.rename(columns={"Yorkshire & Humberside": "Yorkshire and The Humber",
                          "North": "North East"})

df1["South East"] = df1["South East"] + df1["East Anglia"]
df1 = df1.drop(columns="East Anglia")

df2 = clean_pop('data/pop_6.csv')
pop_90 = df2
pop_90.index = pop_90.index.astype('int32').astype('str')
df2["South East"] = df2["South East"] + df2["London"] + df2["East"]
df2 = df2.drop(columns=["London", "East"])

population_regional = pd.concat([df1, df2]).rename({'East': 'East of England'}, axis=1).sort_index(axis=1)
pop_growth_90 = pd.DataFrame(pop_90.iloc[1:].values - pop_90.iloc[:-1].values,
                             index=pop_90.index[1:],
                             columns=pop_90.columns)

builds_regional = pd.DataFrame([clean_builds("data/housebuildingregions_3.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_4.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_5.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_6.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_7.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_8.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_9.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_10.csv").iloc[:, 0],
                                clean_builds("data/housebuildingregions_11.csv").iloc[:, 0]],
                                index=["North East", "North West", "Yorkshire and The Humber",
                                       "East Midlands", "West Midlands", "East of England", "London",
                                       "South East", "South West"]).transpose()
builds_regional = builds_regional.sort_index(axis=1)

In [25]:
r = range(1995, 2018)
rs = [str(i) for i in r]
rs2 = [str(i) for i in range(1978, 2018)]
newR = sales_new.copy().loc[rs]
buildR = builds_regional.copy().loc[rs]
existR = sales_existing.copy().loc[rs]
pg = pop_growth_90.loc[rs].sort_index(axis=1)
newR.name = "New"
existR.name = "Existing"
buildR.name = "Built"
# print(newR.shape)
# print(existR.shape)
# print(pg.shape)
# print(buildR.shape)

In [26]:
pd.DataFrame([sales_new.columns, builds_regional.columns, sales_existing.columns, pg.columns])

,0,1,2,3,4,5,6,7,8
0,East Midlands,East of England,London,North East,North West,South East,South West,West Midlands,Yorkshire and The Humber
1,East Midlands,East of England,London,North East,North West,South East,South West,West Midlands,Yorkshire and The Humber
2,East Midlands,East of England,London,North East,North West,South East,South West,West Midlands,Yorkshire and The Humber
3,East,East Midlands,London,North East,North West,South East,South West,West Midlands,Yorkshire and The Humber


In [27]:
# (pg/buildR).plot(figsize=(16, 8))

In [28]:
def plotL(l):
    (pd.DataFrame([buildR[l], pg[l], newR[l]], index=['build', 'pg', 'new']).transpose()/1000).plot(figsize=(16, 8), title=l)

# plotL('North East')
# plotL('North West')
# plotL('Yorkshire and The Humber')
# plotL('East Midlands')
# plotL('West Midlands')
# plotL('East of England')
# plotL('London')
# plotL('South East')
# plotL('South West')

In [29]:
# (100*pop_growth_90/pop_90.iloc[1:,:]).plot(figsize=(16, 8))